## Project Data Download-Tiingo Data Download
### Context
Machine learning is widely used in financial trading strategy development. Technical data is the most common financial data to development forecasting model in financial industry.
### Content
In this section, I download the technical data for all stocks from NYSE and NASDAQ which are available on Tiingo by using the Tiingo API.
The relevant website is:https://www.tiingo.com/

#### 1. Module initialization

In [2]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime as DT
from datetime import timedelta
import os
from time import sleep
from itertools import chain

#### 2. Initial Setting

In [4]:
os.chdir(r'C:\UCSD_ML_Capstone\01_Input')
StockAll = pd.read_excel('./01_1_DataCodes/AllTickers_20211018.xlsx')
StockTickers  = list(chain(*StockAll.values))
Token2 = 'ddff0c680755cd0a68288e345d4e422e30e2c07b'
TokenList = [Token2]

## Request daily data
token_num = 0
Token = TokenList[token_num]
TotalTime = 0
ErrorTicker = []
headers = { 'Content-Type': 'application/json' }

#### 3. Request Setting

In [5]:
S_time = DT.now()
start_iTicker = 0
i_token = 0
## Avoid data request overwhelming, add some pause points
wait_itoken = 1400
sleepseconds = 60*10

#### 4. Data Request

In [ ]:
for iTicker in range(start_iTicker,len(StockTickers)):
# for iTicker in range(4583,len(StockTickers)):    
    i_token += 1
    if i_token > wait_itoken:
        print("Out of token at " + str(token_num))
        print('Awake at '+ str(DT.now() + timedelta(seconds=sleepseconds)) )
        sleep(sleepseconds)
    
    Ticker = StockTickers[iTicker]
    Loop_start = DT.now()   
    
    try:     # try whether the ticker is available
        try:    # try whether existing historical data
            Current = pd.read_excel("./01_2_RawData/Tiingo_Stock_daily/" + Ticker + "_daily.xlsx")
    
            LastTime = Current.date[len(Current.date)-1]
            LastTime = LastTime[0:10]
            
            reqdata = requests.get("https://api.tiingo.com/tiingo/daily/"+Ticker+"/"+\
                                    "prices?startDate="+LastTime+"&"+\
                                    "columns=open,high,low,close,volume,adjOpen,adjHigh,adjLow,adjClose,adjVolume,divCash,splitFactor&"+\
                                    "token="+Token, headers=headers)
        
            lIntraday = pd.DataFrame(reqdata.json())
            
            NewData = pd.concat([Current, lIntraday], axis = 0)
            NewData = NewData.drop_duplicates(keep ='last')
        
        except FileNotFoundError:
                        
            reqdata = requests.get("https://api.tiingo.com/tiingo/daily/"+Ticker+"/"+\
                               "prices?startDate=2000-01-01&"+\
                                "columns=open,high,low,close,volume,adjOpen,adjHigh,adjLow,adjClose,adjVolume,divCash,splitFactor&"+\
                               "token="+Token, headers=headers)

            NewData = pd.DataFrame(reqdata.json())
        
        sleepseconds2 = 1
        sleep(sleepseconds2)

        # save data
        NewData.to_excel("./01_2_RawData/Tiingo_Stock_daily/" + Ticker + "_daily.xlsx",index=False)

        # calculate the estimated finish time
        Loop_end = DT.now()
        LoopTime = Loop_end-Loop_start
        TotalTime = Loop_end-S_time
        AvgTime = (TotalTime- timedelta(seconds=sleepseconds * ((iTicker-1)//wait_itoken) ) )\
            /(iTicker-start_iTicker+1)
        EstEnd = DT.now() + (len(StockTickers) - iTicker - 1)*AvgTime +\
            timedelta(seconds=sleepseconds * ((len(StockTickers) - iTicker - 1)//wait_itoken) )
        print('Done No.'+ str(iTicker+1) + \
               " " +Ticker+\
              "; Loop time:" + str(LoopTime) +\
              "; Total time:" + str(TotalTime) + \
              "; Average time:" + str(AvgTime) +\
              "; Estimate end:" + str(EstEnd) + '\n'  )

    except ValueError:     # output the unavailable ticker list
        print('Done No.'+ str(iTicker+1) +' '+Ticker + ' is unavailable.')
        ErrorTicker.append(Ticker)
        continue   

#### 5.Outputs

In [ ]:
## Unavailable ticker list
pd.Series(ErrorTicker).to_excel("./01_2_RawData/Tiingo_Stock_daily/" + "ErrorTicker_daily.xlsx", index=False)
## ID of the last run ticker
pd.Series(iTicker).to_excel("./01_2_RawData/Tiingo_Stock_daily/" + "iTicker_daily.xlsx", index=False)